In [22]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

In [23]:
spark = (SparkSession
         .builder
         .master('local[*]')\
         .getOrCreate()
         )

In [24]:
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://minio:9000")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "FFD34B2AC56E76E8BB9E7EFD7D283")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "47A13EFAB1CA1F253FD6F56DEF769")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider","org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
spark._jsc.hadoopConfiguration().set("spark.jars", "postgresql-42.2.14.jar")

In [25]:
# read from minio
df_fake_items = spark.read \
        .format("json") \
        .option("inferSchema", "true") \
        .json("s3a://landing/fake_items//*.json")


In [26]:
df_fake_items.show()

+----------+--------------------+--------------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+-------------------+----------+
|Unnamed: 0|         ai_model_id|      cloned_from_id|      content_filter|content_type|         created_at|dt_current_timestamp|             flavors|                  id|is_archived|              job_id|            metadata|          project_id|    prompt_schema_id|             root_id|    selected_flavors|   status|      step|         updated_at|updated_by|
+----------+--------------------+--------------------+--------------------+------------+-------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+---------

In [30]:
df_fake_items.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- ai_model_id: string (nullable = true)
 |-- cloned_from_id: string (nullable = true)
 |-- content_filter: string (nullable = true)
 |-- content_type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- dt_current_timestamp: long (nullable = true)
 |-- flavors: string (nullable = true)
 |-- id: string (nullable = true)
 |-- is_archived: boolean (nullable = true)
 |-- job_id: string (nullable = true)
 |-- metadata: string (nullable = true)
 |-- project_id: string (nullable = true)
 |-- prompt_schema_id: string (nullable = true)
 |-- root_id: string (nullable = true)
 |-- selected_flavors: string (nullable = true)
 |-- status: string (nullable = true)
 |-- step: string (nullable = true)
 |-- updated_at: string (nullable = true)
 |-- updated_by: string (nullable = true)



In [27]:
df_fake_items.write.format("jdbc")\
    .option("url", "jdbc:postgresql://postgres:5432/postgres") \
    .option("driver", "org.postgresql.Driver").option("dbtable", "demo.fake_items") \
    .option("user", "postgres").option("password", "Sup3rS3c3t").mode('overwrite').save()

In [28]:
spark.stop()